<a href="https://colab.research.google.com/github/vladislavpan1623/-1-/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B04_3c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from diffusers import StableDiffusionPipeline

# Використовуємо базову модель v1.5 (вона найлегша для RAM)
base_model = "runwayml/stable-diffusion-v1-5"
lora_id = "Terapevt1981/mia-lora-01-2807"

print("Завантаження для CPU (може зайняти 1-2 хвилини)...")

# На CPU обов'язково використовуємо torch.float32
pipe = StableDiffusionPipeline.from_pretrained(
    base_model,
    torch_dtype=torch.float32
)
pipe.to("cpu")

# Намагаємося завантажити LoRA
# Якщо виникає помилка розмірів, ми пропускаємо несумісні шари
try:
    pipe.load_lora_weights(lora_id, low_cpu_mem_usage=True)
    print("LoRA завантажена (частково або повністю)!")
except Exception as e:
    print(f"Помилка сумісності: {e}")
    print("Продовжуємо роботу з базовою моделлю v1.5.")

print(" Модель готова до генерації на CPU!")

Завантаження для CPU (може зайняти 1-2 хвилини)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading default_0 was unsuccessful with the following error: 
Error(s) in loading state_dict for UNet2DConditionModel:
	size mismatch for down_blocks.1.attentions.0.transformer_blocks.0.attn2.to_k.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.0.transformer_blocks.0.attn2.to_v.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.1.transformer_blocks.0.attn2.to_k.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.1.transformer_blocks.0.attn2.to_v.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch

Помилка сумісності: Error(s) in loading state_dict for UNet2DConditionModel:
	size mismatch for down_blocks.1.attentions.0.transformer_blocks.0.attn2.to_k.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.0.transformer_blocks.0.attn2.to_v.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.1.transformer_blocks.0.attn2.to_k.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down_blocks.1.attentions.1.transformer_blocks.0.attn2.to_v.lora_A.default_0.weight: copying a param with shape torch.Size([128, 2048]) from checkpoint, the shape in current model is torch.Size([128, 768]).
	size mismatch for down

In [6]:
import ipywidgets as widgets
from IPython.display import display
import mediapy as media

# --- ІНТЕРФЕЙС ---
text_input = widgets.Textarea(
    placeholder='Введіть опис англійською...',
    description='Промпт:',
    layout={'height': '80px', 'width': '80%'}
)
button = widgets.Button(description=" Генерувати", button_style='success')
output = widgets.Output()

def generate_on_cpu(b):
    with output:
        output.clear_output()
        prompt = text_input.value
        if not prompt:
            print("Введіть текст!")
            return

        print(" Почалася генерація на CPU. Будь ласка, зачекайте (2-5 хвилин)...")

        # Оптимізація для CPU
        image = pipe(
            prompt=prompt,
            num_inference_steps=12,  # Мінімум кроків для швидкості
            guidance_scale=7.0,
            width=384,               # Зменшуємо розмір для економії пам'яті
            height=384
        ).images[0]

        media.show_image(image)
        image.save("cpu_result.png")
        print(" Збережено!")

button.on_click(generate_on_cpu)
display(text_input, button, output)

Textarea(value='', description='Промпт:', layout=Layout(height='80px', width='80%'), placeholder='Введіть опис…

Button(button_style='success', description=' Генерувати', style=ButtonStyle())

Output()